In [1]:
from time import sleep
from tqdm import tqdm

In [2]:
import numpy as np
import pandas as pd
import requests

In [3]:
tqdm.pandas()


In [19]:
def to_district(address: str, city: str, district_cache):
    """
    Определяет район и координаты по адресу через OSM Nominatim
    Возвращает dict: {district, lat, lon}
    """

    if not isinstance(address, str) or not address.strip():
        return {
            "district": "Некорректный адрес",
            "lat": None,
            "lon": None
        }

    if not isinstance(city, str) or not city.strip():
        return {
            "district": "Некорректный город",
            "lat": None,
            "lon": None
        }

    url = "https://nominatim.openstreetmap.org/search"

    if city not in district_cache:
        district_cache[city] = {}

    if address in district_cache[city]:
        return district_cache[city][address]

    clean_address = address.replace("корп.", "").replace("корп", "").strip()

    params = {
        "street": clean_address,
        "city": city,
        "state": city,
        "country": "Россия",
        "countrycodes": "ru",
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }

    headers = {
        "User-Agent": "RealEstateDistrictFetcher/1.0 (klvalopatin@edu.hse.ru)",
        "Accept-Language": "ru"
    }

    last_error = None
    for _ in range(3):
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            break
        except Exception as e:
            last_error = e
            sleep(1)
    else:
        result = {
            "district": f"Ошибка запроса: {last_error}",
            "lat": None,
            "lon": None
        }
        district_cache[city][address] = result
        return result

    sleep(0.1)

    if not data:
    
        result = {
            "district": "Адрес не найден",
            "lat": None,
            "lon": None
        }
        district_cache[city][address] = result
        return result

    item = data[0]
    address_info = item.get("address", {})

    if address_info.get("country_code") != "ru" or city not in item.get("display_name", ""):
        result = {
            "district": "Неверный город или страна",
            "lat": None,
            "lon": None
        }
        district_cache[city][address] = result
        return result

    district = (
        address_info.get("city_district") or
        address_info.get("district") or
        address_info.get("borough") or
        address_info.get("municipality") or
        address_info.get("suburb") or
        address_info.get("neighbourhood") or
        address_info.get("quarter") or
        "Район не найден"
    )

    result = {
        "district": district,
        "lat": float(item["lat"]),
        "lon": float(item["lon"])
    }

    district_cache[city][address] = result
    return result


In [11]:
to_district(
    "Конструктора Гуськова, 14 ст2",
    "Москва",
    district_cache
)

{'district': 'Адрес не найден', 'lat': None, 'lon': None}

In [12]:
estate_msk = pd.read_csv('data/estate_msk.csv')
estate_spb = pd.read_csv('data/estate_spb.csv')


In [13]:
cities = {"msk" : "Москва", "spb" : "Санкт-Петербург"}

In [14]:
estate_turple = [
    estate_msk,
    estate_spb
]

In [15]:
estate_msk

,"rent, rub/month",full_address,city,"price_per_sqm, rub/month","square, m^2",type,url
0,1500000,"Пречистенка, 19/11 ст2",Москва,13636,110,Универсальное помещение,https://msk.n1.ru/view/114210381/?open_card_kn
1,1197919,"Таежная, 1",Москва,1917,625,Универсальное помещение,https://msk.n1.ru/view/115999047/?open_card_kn
2,274809,"Дмитровка Б., 32 ст4",Москва,4084,67,Универсальное помещение,https://msk.n1.ru/view/116261376/?open_card_kn
3,386458,"Шумкина, 20 ст1",Москва,1459,265,Универсальное помещение,https://msk.n1.ru/view/116011103/?open_card_kn
4,68125,"Шумкина, 20 ст1",Москва,2084,32,Универсальное помещение,https://msk.n1.ru/view/115777113/?open_card_kn
...,...,...,...,...,...,...,...
7692,1500000,"Краснопролетарская, 36",Москва,3968,378,Помещение под бар/ресторан,https://msk.n1.ru/view/109731158/?open_card_kn
7693,1176000,"Пресненская набережная, 10с",Москва,6461,182,Помещение под бар/ресторан,https://msk.n1.ru/view/111002230/?open_card_kn
7694,2000000,"Пресненская набережная, 10 ст2",Москва,4819,415,Помещение под бар/ресторан,https://msk.n1.ru/view/109683444/?open_card_kn
7695,300003,"Братиславская, 18 корп. 1",Москва,2041,147,Помещение под бар/ресторан,https://msk.n1.ru/view/112349375/?open_card_kn


In [16]:
district_cache = {}

In [20]:
for estate in estate_turple:
    estate['OSM_address'] = estate['full_address'] + ', ' + estate['city'] + ', Россия'
    estate["address_info"] = estate.progress_apply(
        lambda x: to_district(x['full_address'], x['city'], district_cache),
        axis=1
    )
    

100%|██████████| 1730/1730 [17:08<00:00,  1.68it/s]


In [21]:
# Проверяем значение ключа "district" в словаре address_info
estate_msk[
    (estate_msk["address_info"].apply(lambda x: x.get("district")) == 'Адрес не найден') |
    (estate_msk["address_info"].apply(lambda x: x.get("district")) == 'Район не найден')
]

,"rent, rub/month",full_address,city,"price_per_sqm, rub/month","square, m^2",type,url,OSM_address,address_info
0,1500000,"Пречистенка, 19/11 ст2",Москва,13636,110,Универсальное помещение,https://msk.n1.ru/view/114210381/?open_card_kn,"Пречистенка, 19/11 ст2, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
2,274809,"Дмитровка Б., 32 ст4",Москва,4084,67,Универсальное помещение,https://msk.n1.ru/view/116261376/?open_card_kn,"Дмитровка Б., 32 ст4, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
5,320100,"Павелецкий 2-й проезд, 5 ст1",Москва,3000,106,Универсальное помещение,https://msk.n1.ru/view/115502574/?open_card_kn,"Павелецкий 2-й проезд, 5 ст1, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
6,293601,"Мира проспект, 102 ст17",Москва,2667,110,Универсальное помещение,https://msk.n1.ru/view/114858057/?open_card_kn,"Мира проспект, 102 ст17, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
11,800000,"Масловка Верхн., 20 ст1",Москва,2817,283,Универсальное помещение,https://msk.n1.ru/view/111240795/?open_card_kn,"Масловка Верхн., 20 ст1, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
...,...,...,...,...,...,...,...,...,...
7662,990000,"Мневники Нижн., 9",Москва,4977,198,Помещение под бар/ресторан,https://msk.n1.ru/view/112711743/?open_card_kn,"Мневники Нижн., 9, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
7671,600000,"Земляной Вал, 50а ст3",Москва,4123,145,Помещение под бар/ресторан,https://msk.n1.ru/view/115312443/?open_card_kn,"Земляной Вал, 50а ст3, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
7674,649000,"Огородный проезд, 16/1 ст6",Москва,4013,161,Помещение под бар/ресторан,https://msk.n1.ru/view/115306316/?open_card_kn,"Огородный проезд, 16/1 ст6, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
7678,135000,"пос. Коммунарка, 14",Москва,2020,66,Помещение под бар/ресторан,https://msk.n1.ru/view/114593131/?open_card_kn,"пос. Коммунарка, 14, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."


In [22]:
estate_cities = pd.concat([
    estate_msk,
    estate_spb
])

In [23]:
def to_normAdress(address: str):
    return address.split(',')[0]


In [24]:
mask = (estate_cities["address_info"].apply(lambda x: x.get("district")) == 'Адрес не найден')

estate_cities.loc[mask, "address_info"] = estate_cities.loc[mask].progress_apply(
    lambda x: to_district(to_normAdress(x["full_address"]), x["city"], district_cache),
    axis=1
)


100%|██████████| 1746/1746 [08:03<00:00,  3.61it/s]


In [25]:
estate_cities[
    (estate_cities["address_info"].apply(lambda x: x.get("district")) == 'Адрес не найден') |
    (estate_cities["address_info"].apply(lambda x: x.get("district")) == 'Район не найден')
]

,"rent, rub/month",full_address,city,"price_per_sqm, rub/month","square, m^2",type,url,OSM_address,address_info
2,274809,"Дмитровка Б., 32 ст4",Москва,4084,67,Универсальное помещение,https://msk.n1.ru/view/116261376/?open_card_kn,"Дмитровка Б., 32 ст4, Москва, Россия","{'district': 'Район не найден', 'lat': 55.1728..."
11,800000,"Масловка Верхн., 20 ст1",Москва,2817,283,Универсальное помещение,https://msk.n1.ru/view/111240795/?open_card_kn,"Масловка Верхн., 20 ст1, Москва, Россия","{'district': 'Адрес не найден', 'lat': None, '..."
14,358800,"проезд улицашоссепр-дпр-д-й, 4 ст4",Москва,1560,230,Универсальное помещение,https://msk.n1.ru/view/114562535/?open_card_kn,"проезд улицашоссепр-дпр-д-й, 4 ст4, Москва, Ро...","{'district': 'Адрес не найден', 'lat': None, '..."
18,87600,"проезд улицашоссепр-дпр-д-й, 4 ст4",Москва,2400,36,Универсальное помещение,https://msk.n1.ru/view/114547323/?open_card_kn,"проезд улицашоссепр-дпр-д-й, 4 ст4, Москва, Ро...","{'district': 'Адрес не найден', 'lat': None, '..."
20,175350,"Летчика Грицевца (п Внуковское), 5",Москва,1500,116,Универсальное помещение,https://msk.n1.ru/view/113087033/?open_card_kn,"Летчика Грицевца (п Внуковское), 5, Москва, Ро...","{'district': 'Адрес не найден', 'lat': None, '..."
...,...,...,...,...,...,...,...,...,...
1664,182400,"проспект Старорусский, 12",Санкт-Петербург,1500,121,Помещение под бар/ресторан,https://spb.n1.ru/view/116105509/?open_card_kn,"проспект Старорусский, 12, Санкт-Петербург, Ро...","{'district': 'Район не найден', 'lat': 59.8066..."
1694,299300,"проспект Средний Васильевского острова, 14/45",Санкт-Петербург,1088,275,Помещение под бар/ресторан,https://spb.n1.ru/view/115734209/?open_card_kn,"проспект Средний Васильевского острова, 14/45,...","{'district': 'Адрес не найден', 'lat': None, '..."
1696,139077,"Заречная, 54",Санкт-Петербург,1530,90,Помещение под бар/ресторан,https://spb.n1.ru/view/113624912/?open_card_kn,"Заречная, 54, Санкт-Петербург, Россия","{'district': 'Район не найден', 'lat': 60.0838..."
1704,1424000,"Витебский проспект, 99 корп. 1",Санкт-Петербург,2000,712,Помещение под бар/ресторан,https://spb.n1.ru/view/110466463/?open_card_kn,"Витебский проспект, 99 корп. 1, Санкт-Петербур...","{'district': 'Район не найден', 'lat': 59.8146..."


In [26]:
estate_cities["district"] = estate_cities["address_info"].apply(
    lambda x: x.get("district") if isinstance(x, dict) else None
)
estate_cities["lat"] = estate_cities["address_info"].apply(
    lambda x: x.get("lat") if isinstance(x, dict) else None
)
estate_cities["lon"] = estate_cities["address_info"].apply(
    lambda x: x.get("lon") if isinstance(x, dict) else None
)

In [27]:
estate_cities = estate_cities.reset_index(drop=True)

In [32]:
estate_cities.to_csv("data/estate_cities.csv", index=False)

In [33]:
estate_cities

,"rent, rub/month",full_address,city,"price_per_sqm, rub/month","square, m^2",type,url,OSM_address,address_info,district,lat,lon
0,1500000,"Пречистенка, 19/11 ст2",Москва,13636,110,Универсальное помещение,https://msk.n1.ru/view/114210381/?open_card_kn,"Пречистенка, 19/11 ст2, Москва, Россия","{'district': 'район Хамовники', 'lat': 55.7430...",район Хамовники,55.743015,37.596967
1,1197919,"Таежная, 1",Москва,1917,625,Универсальное помещение,https://msk.n1.ru/view/115999047/?open_card_kn,"Таежная, 1, Москва, Россия","{'district': 'Лосиноостровский район', 'lat': ...",Лосиноостровский район,55.881107,37.700736
2,274809,"Дмитровка Б., 32 ст4",Москва,4084,67,Универсальное помещение,https://msk.n1.ru/view/116261376/?open_card_kn,"Дмитровка Б., 32 ст4, Москва, Россия","{'district': 'Район не найден', 'lat': 55.1728...",Район не найден,55.172889,37.122824
3,386458,"Шумкина, 20 ст1",Москва,1459,265,Универсальное помещение,https://msk.n1.ru/view/116011103/?open_card_kn,"Шумкина, 20 ст1, Москва, Россия","{'district': 'район Сокольники', 'lat': 55.789...",район Сокольники,55.789422,37.666981
4,68125,"Шумкина, 20 ст1",Москва,2084,32,Универсальное помещение,https://msk.n1.ru/view/115777113/?open_card_kn,"Шумкина, 20 ст1, Москва, Россия","{'district': 'район Сокольники', 'lat': 55.789...",район Сокольники,55.789422,37.666981
...,...,...,...,...,...,...,...,...,...,...,...,...
9422,139000,"Московский проспект, 72 корп. 2",Санкт-Петербург,2249,61,Помещение под бар/ресторан,https://spb.n1.ru/view/106961445/?open_card_kn,"Московский проспект, 72 корп. 2, Санкт-Петербу...","{'district': 'округ Измайловское', 'lat': 59.9...",округ Измайловское,59.902811,30.318498
9423,961538,"Энергетиков проспект, 6",Санкт-Петербург,3083,311,Помещение под бар/ресторан,https://spb.n1.ru/view/110365219/?open_card_kn,"Энергетиков проспект, 6, Санкт-Петербург, Россия","{'district': 'округ Большая Охта', 'lat': 59.9...",округ Большая Охта,59.955832,30.432095
9424,110000,"Заозерная, 3 корп. 3",Санкт-Петербург,2444,45,Помещение под бар/ресторан,https://spb.n1.ru/view/112083708/?open_card_kn,"Заозерная, 3 корп. 3, Санкт-Петербург, Россия","{'district': 'округ Московская застава', 'lat'...",округ Московская застава,59.906229,30.324407
9425,518960,"Заречная, 52 корп. 1",Санкт-Петербург,2600,199,Помещение под бар/ресторан,https://spb.n1.ru/view/113137877/?open_card_kn,"Заречная, 52 корп. 1, Санкт-Петербург, Россия","{'district': 'Торфяное', 'lat': 60.082999, 'lo...",Торфяное,60.082999,30.342624


In [44]:
mask = (estate_cities['city'] == 'Санкт-Петербург') & (estate_cities['district'] == 'Адрес не найден')
estate_cities.loc[mask, :].shape

(55, 12)